In [1]:
import csv
import os
import re
import unicodedata
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Chrome Options
options = Options()
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1920,1080")
options.add_argument("--headless")  # Headless mode for speed

# ChromeDriver path
path = r'F:\SaylaniPython\Selenium Driver\chromedriver-win64\chromedriver.exe'
service = Service(executable_path=path)

# CSV File Path
csv_file = "tradingview_stock_data.csv"

def clean_text(text):
    """Normalize and remove non-alphanumeric characters while replacing unwanted symbols with '-'"""
    text = unicodedata.normalize("NFKD", text)  # Normalize Unicode text
    text = text.encode("ascii", "ignore").decode("utf-8")  # Remove special Unicode characters
    text = re.sub(r'[^a-zA-Z0-9.%+-]', '-', text.strip())  # Allow numbers, alphabets, %, and signs
    return text

# Initialize WebDriver
driver = webdriver.Chrome(service=service, options=options)
driver.get("https://www.tradingview.com/markets/stocks-usa/market-movers-all-stocks/")

# Wait for anchor links to load
WebDriverWait(driver, 20).until(
    EC.presence_of_element_located((By.CLASS_NAME, "tickerNameBox-GrtoTeat"))
)

# Find all anchor links
stock_links = driver.find_elements(By.XPATH, "//a[contains(@class, 'tickerNameBox-GrtoTeat')]")

# Create CSV file if not exists & write headers
file_exists = os.path.exists(csv_file)
with open(csv_file, mode="a", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    if not file_exists:
        writer.writerow(["Serial No", "Stock Symbol", "Metric1", "Metric2", "Metric3", "..."])


index = 1

while True:
    stock_links = driver.find_elements(By.XPATH,"//a[contains(@class, 'tickerNameBox-GrtoTeat')]")

    if not stock_links:
        print("No Stock Link Found Exist")
        break

    # Scrape stocks dynamically
    for link in stock_links:
        stock_symbol = clean_text(link.text.strip())
        stock_url = link.get_attribute("href")

        print(f"Scraping stock {index}: {stock_symbol} - {stock_url}")

        # Open stock page in new tab
        driver.execute_script("window.open(arguments[0]);", stock_url)
        driver.switch_to.window(driver.window_handles[-1])

        time.sleep(3)  # Allow page to load

        try:
            # Wait for headings & data to load
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CLASS_NAME, "label-QCJM7wcY"))
            )
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CLASS_NAME, "wrapper-QCJM7wcY"))
            )

            # Extract headings & data (first 12)
            headings_elements = driver.find_elements(By.CLASS_NAME, "label-QCJM7wcY")
            headings = [h.text.strip() for h in headings_elements[:12]]

            data_elements = driver.find_elements(By.CLASS_NAME, "wrapper-QCJM7wcY")
            data_values = [clean_text(d.text.strip()) for d in data_elements[:12]]

            # Ensure data mapping is correct
            if len(headings) != len(data_values):
                print(f"Data mismatch on {stock_symbol}, skipping...")
                driver.close()
                driver.switch_to.window(driver.window_handles[0])
                continue

            # **Save data immediately in CSV**
            with open(csv_file, mode="a", newline="", encoding="utf-8") as file:
                writer = csv.writer(file)
                writer.writerow([index, stock_symbol] + data_values)

            print(f"Data for {stock_symbol} saved!")
            index += 1

        except Exception as e:
            print(f"Error scraping {stock_symbol}: {e}")

        # Close tab and switch back
        driver.close()
        driver.switch_to.window(driver.window_handles[0])

    # **Check for Load More button**
    try:
        load_more_btn = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.XPATH, "//span[contains(text(), 'Load More')]"))
        )
        print("Clicking 'Load More' to fetch more stocks...")
        load_more_btn.click()
        time.sleep(5)  # Wait for data to load
    except:
        print("No 'Load More' button found, scraping completed!")
        break  # Stop scraping when Load More disappears

driver.quit()
print(f"All stock data successfully stored in {csv_file}")

Scraping stock 1: A - https://www.tradingview.com/symbols/NYSE-A/
Data for A saved!
Scraping stock 2: AA - https://www.tradingview.com/symbols/NYSE-AA/
Data for AA saved!
Scraping stock 3: AACB - https://www.tradingview.com/symbols/NASDAQ-AACB/
Data for AACB saved!
Scraping stock 4: AACT - https://www.tradingview.com/symbols/NYSE-AACT/
Data for AACT saved!
Scraping stock 5: AAL - https://www.tradingview.com/symbols/NASDAQ-AAL/
Data for AAL saved!
Scraping stock 6: AAM - https://www.tradingview.com/symbols/NYSE-AAM/
Data for AAM saved!
Scraping stock 7: AAME - https://www.tradingview.com/symbols/NASDAQ-AAME/
Data for AAME saved!
Scraping stock 8: AAMI - https://www.tradingview.com/symbols/NYSE-AAMI/
Data for AAMI saved!
Scraping stock 9: AAOI - https://www.tradingview.com/symbols/NASDAQ-AAOI/
Data for AAOI saved!
Scraping stock 10: AAON - https://www.tradingview.com/symbols/NASDAQ-AAON/
Data for AAON saved!
Scraping stock 11: AAP - https://www.tradingview.com/symbols/NYSE-AAP/
Data for 

KeyboardInterrupt: 